In [1]:
import os
import numpy as np
import pickle as pkl
import nltk
from nltk.tokenize import RegexpTokenizer
import glob

In [2]:
data_dir = '/data/unagi0/ktokitake/encdecmodel/coco'
pkl_path = os.path.join(data_dir, 'captions.pickle')

In [3]:
with open(pkl_path, 'rb') as f:
    captions = pkl.load(f)

In [8]:
nltk.download('averaged_perceptron_tagger')
filename_to_captions = {}
all_files = glob.glob(os.path.join(data_dir, 'text/train2014/*.txt'))
for i,f in enumerate(all_files):
    if i%1000:
        print('processing ', i)
    basename = os.path.basename(f)
    basename = basename[:-4]
    with open(f, "r") as f:
        new_captions = []
        captions = f.read().split('\n')
        for cap in captions:
            if len(cap)==0:
                continue
            cap = cap.replace('\ufffd\ufffd', ' ')
            tokenizer = RegexpTokenizer(r'\w+')
            tokens = tokenizer.tokenize(cap.lower())
            
            if len(tokens)==0:
                continue
            
            new_cap = []
            for t in tokens:
                t = t.encode('ascii', 'ignore').decode('ascii')
                if len(t)==0:
                    continue
                new_cap.append(t)
            
            new_captions.append(new_cap)
        filename_to_captions[basename] = new_captions
        
            

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/mil/tokitake/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


KeyboardInterrupt: 

In [17]:
### load glove vector ###
home_dir = '/home/mil/tokitake'
glove_path = os.path.join(home_dir, 'glove/glove.6B.300d.txt')
words = []
embeddings = []
with open(glove_path, "r" ) as f:
    for line in f.readlines():
        line = line.strip().split()
        word = line[0]
        emb = list(map(float, line[1:]))
        words.append(word)
        embeddings.append(emb)

In [26]:
word = 'water'
idx = words.index(word)
target = np.asarray(embeddings[idx])
embeddings = np.array(embeddings)
dots = np.dot(embeddings, target)

words_np = np.array(words)

indices = np.argsort(dots)
indices = indices[::-1]

rel_words = words_np[indices[:10]]
for word in rel_words:
    print(word)

water
sewage
irrigation
river
drinking
electricity
groundwater
drainage
pollution
waters


[ 8.32840034  7.79653467  6.58307392  7.76880859  5.36510149  7.0389635
 10.1682455   9.45187988  8.32243243  6.91407709]


In [ ]:
all_keys = list(filename_to_captions.keys())
